In [2]:
# Ali Babolhavaeji 
# 7/5/2019

import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F

from torchvision import datasets, transforms, models

from torch.utils.data import *
from PIL import Image
import numpy as np

In [3]:
#splitting data
def split_image_data(train_data,
                     test_data=None,
                     batch_size=20,
                     num_workers=0,
                     valid_size=0.2,
                     sampler=SubsetRandomSampler):
    
    num_train=len(train_data)
    indices= list(range(num_train))
    np.random.shuffle(indices)
    split=int(np.floor(valid_size*num_train))
#     print(split)
    train_idx,valid_idx=indices[split:],indices[:split]
    train_sampler=sampler(train_idx)
    valid_sampler=sampler(valid_idx)
    
    if test_data is not None:
        test_loader= DataLoader(test_data, batch_size=batch_size,
                                num_workers=num_workers) 
    else:
        train_idx,test_idx= train_idx[split:],train_idx[:split]
        train_sampler=sampler(train_idx)
        test_sampler =sampler(test_idx)

        test_loader= DataLoader(train_data, batch_size=batch_size,
                                 num_workers=num_workers,
                                 sampler=test_sampler)
        
    train_loader= DataLoader(train_data, batch_size=batch_size,
                             num_workers=num_workers,
                             sampler=train_sampler)
    
    valid_loader= DataLoader(train_data,batch_size=batch_size,
                             num_workers=num_workers, 
                             sampler=valid_sampler)
    
    return train_loader, valid_loader , test_loader
#     return train_loader , test_loader
        

In [5]:
batch_size = 50
cifar10_mean = [0.4915, 0.4823, 0.4468]
cifar10_std  = [0.2470, 0.2435, 0.2616] 
train_transform = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(10),
                                      transforms.ToTensor(),
                                      transforms.Normalize(cifar10_mean, cifar10_std)
                                     ])

test_transform = transforms.Compose([transforms.Resize((224,224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(cifar10_mean, cifar10_std)
                                    ])

train_data = datasets.CIFAR10('Cifar10', train=True,
                              download=False, transform=train_transform)
test_data = datasets.CIFAR10('Cifar10', train=False,
                             download=False, transform=test_transform)

trainloader,validloader,testloader = split_image_data(train_data,test_data,batch_size=batch_size)

len(trainloader),len(testloader),len(validloader)

# toTensor() converts a numpy array to a PyTorch Tensor (all our images are constructed as numpy arrays by the 
# dataset class when read from disk).
# normalize() is a transform that normalizes according to the passed values
# of means and STD of each channel as separate lists or tuples.

(800, 200, 200)

In [71]:
# build the core classes to implement a basic neural network by using Pytorch
import time as time
from collections import defaultdict
import math

def update_classwise_accuracies(preds,labels,class_correct,class_totals):
    correct=np.squeeze(preds.eq(labels.data.view_as(preds)))
    for i in range(lables.shape[0]):
        label=label.data[i].item()
        class_correct[label] += correct[i]+item()
        class_totals[label] +=1
        
def get_accuracies(class_names,class_correct,class_totals):

    accuracy = (100*np.sum(list(class_correct.values()))/np.sum(list(class_totals.values())))
    class_accuracies = [(class_names[i],100.0*(class_correct[i]/class_totals[i]))
                        for i in class_names.keys() if class_totals[i] > 0]
    return accuracy,class_accuracies
    

class Network(nn.Module):
    def __init__(self,device=None):
        super().__init__()
        if device is not None:
            self.device=device
        else:
            self.device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.best_accuracy = 0.
            
    def forward(self,x):
        pass

                
    def train_(self,trainloader,criterion,optimizer,print_every):
        self.train()
        t0=time.time()
        batches = 0
        running_loss = 0
        for inputs, labels in trainloader:
            batches += 1
            #t1 = time.time()

            inputs, labels = inputs.to(self.device), labels.to(self.device)
            optimizer.zero_grad()
            outputs = self.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            loss = loss.item()
            #print('training this batch took {:.3f} seconds'.format(time.time() - t1))
            running_loss += loss
            if batches % print_every == 0:
                print(f"{time.asctime()}.."
                        f"Time Elapsed = {time.time()-t0:.3f}.."
                        f"Batch {batches+1}/{len(trainloader)}.. "
                        f"Average Training loss: {running_loss/(batches):.3f}.. "
                        f"Batch Training loss: {loss:.3f}.. "
                        )
                t0 = time.time()
                return running_loss/len(trainloader)
            
            
    def validate_(self,validloader):
        running_loss = 0.
        accuracy = 0
        class_correct = defaultdict(int)
        class_totals = defaultdict(int)
        self.eval()
        with torch.no_grad():
            for inputs, labels in validloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.forward(inputs)
                loss = self.criterion(outputs, labels)
                running_loss += loss.item()
                _, preds = torch.max(torch.exp(outputs), 1)
                update_classwise_accuracies(preds,labels,class_correct,class_totals)
                accuracy = (100*np.sum(list(class_correct.values()))/np.sum(list(class_totals.values())))
                self.train()
                return (running_loss/len(validloader),accuracy)
            
        def evaluate(self,testloader):
            self.eval()
            self.model.to(self.device)
            class_correct = defaultdict(int)
            class_totals = defaultdict(int)
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)
                    outputs = self.forward(inputs)
                    ps = torch.exp(outputs)
                    _, preds = torch.max(ps, 1)
                    update_classwise_accuracies(preds,labels,class_correct,class_totals)

            self.train()
            return get_accuracies(self.class_names,class_correct,class_totals)
       
    
        def predict(self,inputs,topk=1):
            self.eval()
            self.model.to(self.device)
            with torch.no_grad():
                inputs = inputs.to(self.device)
                outputs = self.forward(inputs)
                ps = torch.exp(outputs)
                p,top = ps.topk(topk, dim=1)
            return p,top

    def fit(self,trainloader,validloader,epochs=2,print_every=10,validate_every=1):

        for epoch in range(epochs):
            self.model.to(self.device)
            print('epoch {:3d}/{}'.format(epoch+1,epochs))
            epoch_train_loss =  self.train_(trainloader,self.criterion,
                                            self.optimizer,print_every)

            if  validate_every and (epoch % validate_every == 0):
                t2 = time.time()
                epoch_validation_loss,epoch_accuracy = self.validate_(validloader)
                time_elapsed = time.time() - t2
                print(f"{time.asctime()}--Validation time {time_elapsed:.3f} seconds.."
                      f"Epoch {epoch+1}/{epochs}.. "
                      f"Epoch Training loss: {epoch_train_loss:.3f}.. "
                      f"Epoch validation loss: {epoch_validation_loss:.3f}.. "
                      f"validation accuracy: {epoch_accuracy:.3f}")
                if self.best_accuracy == 0. or (epoch_accuracy > self.best_accuracy):
                    print('updating best accuracy: previous best = {:.3f} new best = {:.3f}'.format(self.best_accuracy,
                                                                                     epoch_accuracy))
                    self.best_accuracy = epoch_accuracy
                    torch.save(self.state_dict(),self.best_accuracy_file)

                self.train() # just in case we forgot to put the model back to train mode in validate

        print('loading best accuracy model')
        self.load_state_dict(torch.load(self.best_accuracy_file))

    def set_criterion(self,criterion_name):
            if criterion_name.lower() == 'nllloss':
                self.criterion_name = 'NLLLoss'
                self.criterion = nn.NLLLoss()
            elif criterion_name.lower() == 'crossentropyloss':
                self.criterion_name = 'CrossEntropyLoss'
                self.criterion = nn.CrossEntropyLoss()
        
        
    def set_optimizer(self,params,optimizer_name='adam',lr=0.003):
        from torch import optim

        if optimizer_name.lower() == 'adam':
            print('setting optim Adam')
            self.optimizer = optim.Adam(params,lr=lr)
            self.optimizer_name = optimizer_name
        elif optimizer.lower() == 'sgd':
            print('setting optim SGD')
            self.optimizer = optim.SGD(params,lr=lr)

        elif optimizer.lower() == 'adadelta':
            print('setting optim Ada Delta')
            self.optimizer = optim.Adadelta(params)
            
    def set_model_params(self,
                         criterion_name,
                         optimizer_name,
                         lr, # learning rate
                         dropout_p,
                         model_name,
                         best_accuracy,
                         best_accuracy_file,
                         chkpoint_file):

        self.criterion_name = criterion_name
        self.set_criterion(criterion_name)
        self.optimizer_name = optimizer_name
        self.set_optimizer(self.parameters(),optimizer_name,lr=lr)
        self.lr = lr
        self.dropout_p = dropout_p
        self.model_name =  model_name
        self.best_accuracy = best_accuracy
        print('set_model_params: best accuracy = {:.3f}'.format(self.best_accuracy))
        self.best_accuracy_file = best_accuracy_file
        self.chkpoint_file = chkpoint_file

    def get_model_params(self):
        params = {}
        params['device'] = self.device
        params['model_name'] = self.model_name
        params['optimizer_name'] = self.optimizer_name
        params['criterion_name'] = self.criterion_name
        params['lr'] = self.lr
        params['dropout_p'] = self.dropout_p
        params['best_accuracy'] = self.best_accuracy
        print('get_model_params: best accuracy = {:.3f}'.format(self.best_accuracy))
        params['best_accuracy_file'] = self.best_accuracy_file
        params['chkpoint_file'] = self.chkpoint_file
        print('get_model_params: chkpoint file = {}'.format(self.chkpoint_file))
        return params

    def save_chkpoint(self):
        saved_model = {}
        saved_model['params'] = self.get_model_params()
        torch.save(saved_model,self.chkpoint_file)
        print('checkpoint created successfully in {}'.format(self.chkpoint_file))
            


In [117]:
class FC(Network):
    def __init__(self,num_inputs,
                     num_outputs,
                     layers=[],
                     lr=0.003,
                     class_names=None,
                     optimizer_name='Adam',
                     dropout_p=0.2,
                     non_linearity='relu',
                     criterion_name='NLLLoss',
                     model_type='classifier',
                     best_accuracy=0.,
                     best_accuracy_file ='best_accuracy.pth',
                     chkpoint_file ='chkpoint_file.pth',
                     device=None):
        super().__init__(device=device)

        self.set_model_params(criterion_name,
                                  optimizer_name,
                                  lr,
                                  dropout_p,
                                  'FC',
                                  best_accuracy,
                                  best_accuracy_file,
                                  chkpoint_file
                                  )
        self.non_linearity = non_linearity
        self.model = nn.Sequential()
        
        if len(layers) > 0:
            self.model.add_module('fc1',nn.Linear(num_inputs,layers[0]))
            self.model.add_module('relu1',nn.ReLU())
            self.model.add_module('dropout1',nn.Dropout(p=dropout_p,inplace=True))
            for i in range(1,len(layers)):
                self.model.add_module('fc'+str(i+1),nn.Linear(layers[i-1],layers[i]))
                self.model.add_module('relu'+str(i+1),nn.ReLU())
                self.model.add_module('dropout'+str(i+1),nn.Dropout(p=dropout_p)) 
            self.model.add_module('out',nn.Linear(layers[-1],num_outputs))

                                                                                                      
        else:
            self.model.add_module('out',nn.Linear(num_inputs,num_outputs))

In [109]:
train_data = datasets.MNIST(root='data',download=True,
                            transform = transforms.transforms.ToTensor())

train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(10),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.0839, 0.2038, 0.1042],[0.2537, 0.3659, 0.2798])
                                     ])

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize([0.0839, 0.2038, 0.1042],[0.2537, 0.3659, 0.2798])
                                    ])


In [110]:
train_dataset = datasets.MNIST(root='data',download=False,train=True, transform = train_transform)
test_dataset = datasets.MNIST(root='data',download=False,train=False,transform = test_transform)

In [111]:
trainloader,validloader,testloader = split_image_data(train_dataset,test_dataset,batch_size=50)
len(trainloader),len(validloader),len(testloader)

(960, 240, 200)

In [116]:
net =  FC(num_inputs=784,
          num_outputs=10,
          layers=[512,512],
          optimizer_name='Adadelta',
          best_accuracy_file ='best_accuracy_mnist_fc_test.pth',
          chkpoint_file ='chkpoint_file_mnist_fc_test.pth')
setting optim Ada Delta

SyntaxError: invalid syntax (<ipython-input-116-29a14200b22b>, line 7)